# Train an Object Detection model using Mediapipe

This notebook trains a model on the [Bakery Dataset from Roboflow](https://universe.roboflow.com/project-hxki1/pr_bakery/dataset/1) using [Google Mediapipe Model Maker](https://ai.google.dev/edge/mediapipe/solutions/model_maker).

We followed [this guide](https://ai.google.dev/edge/mediapipe/solutions/customization/object_detector) to learn how to train models using custom data.

## Step 01: Initial Setup

This involves installing all the dependencies required and importing the required Python classes. We followed the [setup guide](https://ai.google.dev/edge/mediapipe/solutions/setup_python) provided by Google in order to accomplish this. We faced quite a few dependency issues here and you have to use [a very specific config](https://github.com/google-ai-edge/mediapipe/issues/5214#issuecomment-2334011576) for it to work. However, this config will also introduce issues that are hard to solve so this notebook should ideally be executed in [Google Colab](https://colab.research.google.com/).

```bash
python --version
pip install --upgrade pip
pip install mediapipe-model-maker
```

In [ ]:
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

## Step 02: Prepare Data

Retraining a model for object detection requires a dataset containing the items or classes you want the final model to recognize. As mentioned earlier, we are using the [Bakery Dataset from Roboflow](https://universe.roboflow.com/project-hxki1/pr_bakery/dataset/1). However, this dataset cannot be used directly. We need to ensure the file format is compatible with what Google MediaPipe requires for its Model Maker to function. 

The dataset is in COCO format, and we made additional modifications to the file locations and names to align with [MediaPipe's sample dataset](https://ai.google.dev/edge/mediapipe/solutions/customization/object_detector#prepare_data). 

In [ ]:
!unzip bakery_dataset.zip
train_dataset_path = "bakery_dataset/train"
validation_dataset_path = "bakery_dataset/validation"

Archive:  bakery_dataset.zip
   creating: bakery_dataset/
  inflating: bakery_dataset/README.roboflow.txt  
  inflating: __MACOSX/bakery_dataset/._README.roboflow.txt  
  inflating: bakery_dataset/README.dataset.txt  
  inflating: __MACOSX/bakery_dataset/._README.dataset.txt  
  inflating: bakery_dataset/.DS_Store  
  inflating: __MACOSX/bakery_dataset/._.DS_Store  
   creating: bakery_dataset/test/
  inflating: __MACOSX/bakery_dataset/._test  
   creating: bakery_dataset/train/
   creating: bakery_dataset/validation/
  inflating: bakery_dataset/test/_annotations.coco.json  
  inflating: __MACOSX/bakery_dataset/test/.__annotations.coco.json  
  inflating: bakery_dataset/test/pic0339_jpg.rf.57eafc4b299a4c1098ea8812203120cd.jpg  
  inflating: __MACOSX/bakery_dataset/test/._pic0339_jpg.rf.57eafc4b299a4c1098ea8812203120cd.jpg  
  inflating: bakery_dataset/test/pic0213_jpg.rf.b19a76af6b76dfc87b496f77b196b4a9.jpg  
  inflating: __MACOSX/bakery_dataset/test/._pic0213_jpg.rf.b19a76af6b76dfc87b

## Step 03: Review dataset

Verify the dataset content by printing the categories from the labels.json file. There should be 3 total categories. Index 0 is always set to be the background class which may be unused in the dataset.

In [ ]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

0: bakery
1: Bread
2: Cake
3: Croissant
4: Puff Pastry


Update the labels.json file to ensure 'background' is at index 0.

In [ ]:
import json
import os

def update_labels_json(json_file_path):

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  background_exists = False
  for category in data['categories']:
    if category['id'] == 0 and category['name'] == 'background':
      background_exists = True
      break

  if not background_exists:
    data['categories'].insert(0, {'id': 0, 'name': 'background'})

    for i in range(1, len(data['categories'])):
      data['categories'][i]['id'] = i

    for annotation in data['annotations']:
      if annotation['category_id'] != 0:  
        annotation['category_id'] = annotation['category_id'] +1 

  with open(json_file_path, 'w') as f:
    json.dump(data, f, indent=2)

update_labels_json(os.path.join(train_dataset_path, "labels.json"))
update_labels_json(os.path.join(validation_dataset_path, "labels.json"))

## Step 04: Create Dataset

In [ ]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

train_data size:  591
validation_data size:  12


## Step 05: Retraining the model

In this code, we're preparing to **retrain a MediaPipe Object Detector model** by specifying the model type (`MOBILENET_MULTI_AVG`), hyperparameters (e.g., export directory), and configuration options.

MediaPipe uses the term "retraining" because it fine-tunes pre-trained models (like MobileNet) to adapt them to your custom dataset, leveraging existing knowledge to efficiently specialize the model for new classes or tasks without starting from scratch. 

This approach is faster and works well for specific use cases like detecting bakery items.

In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


73/73 [==============================] - 290s 3s/step - total_loss: 2.4132 - cls_loss: 2.0767 - box_loss: 0.0055 - model_loss: 2.3503 - val_total_loss: 1.4088 - val_cls_loss: 1.1569 - val_box_loss: 0.0038 - val_model_loss: 1.3459
Epoch 2/30
73/73 [==============================] - 221s 3s/step - total_loss: 1.2632 - cls_loss: 1.0245 - box_loss: 0.0035 - model_loss: 1.2003 - val_total_loss: 1.1166 - val_cls_loss: 0.8955 - val_box_loss: 0.0032 - val_model_loss: 1.0537
Epoch 3/30
73/73 [==============================] - 230s 3s/step - total_loss: 0.8599 - cls_loss: 0.6826 - box_loss: 0.0023 - model_loss: 0.7970 - val_total_loss: 0.8773 - val_cls_loss: 0.6596 - val_box_loss: 0.0031 - val_model_loss: 0.8144
Epoch 4/30
73/73 [==============================] - 225s 3s/step - total_loss: 0.6057 - cls_loss: 0.4646 - box_loss: 0.0016 - model_loss: 0.5428 - val_total_loss: 0.7748 - val_cls_loss: 0.5480 - val_box_loss: 0.0033 - val_model_loss: 0.7119
Epoch 5/30
73/73 [=============================

## Step 06: Evaluating the model performance

After training the model, we evaluate it on validation dataset and print the `loss` and `coco_metrics`. The most important metric for evaluating the model performance is typically the `AP` coco metric for Average Precision.

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

3/3 [==============================] - 3s 616ms/step - total_loss: 0.7368 - cls_loss: 0.4987 - box_loss: 0.0035 - model_loss: 0.6740
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.652
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.794
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.728
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.225
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.736
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.582
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.699
 Average Recall     (AR) @[ I

This output shows the results from evaluating an object detection model. Here’s a breakdown of what the numbers mean:

### 1. Losses

   - **Total Loss:** 0.7368 — Overall model performance.
   - **Classification Loss (cls_loss):** 0.4987 — How well the model classifies objects.
   - **Bounding Box Loss (box_loss):** 0.0035 — How accurately the model locates objects.
   - **Model Loss:** 0.6740 — A summary of the model’s errors.

### 2. Model Performance

   - **AP (Average Precision):**
     - **Overall:** 0.652 — The model is fairly accurate at detecting objects.
     - **At IoU 0.50 (less strict matching):** 0.794 — The model performs better with lower matching thresholds.
     - **At IoU 0.75 (more strict matching):** 0.728 — The model does well with medium-to-high precision.
   - **Recall:**
     - **Overall:** 0.705 — The model detects most of the objects, but it’s less accurate with smaller objects.
     - **For Large Objects:** 0.785 — Great performance with large objects.

### 3. Strengths & Weaknesses

   - **Strengths:** The model performs well with large and medium-sized objects, with high precision (AP50) and recall.
   - **Weaknesses:** Struggles with small objects, as the AP and recall for small objects are very low (AP = -1).

### Conclusion

This model is quite good for detecting medium and large objects but needs improvement for small objects. Overall, it performs well in real-world scenarios with a decent AP score of 0.652 and recall of 0.705.

## Step 07: Export model

After creating the model, we convert and export it to a Tensorflow Lite model format for later use on an on-device application. The export also includes model metadata, which includes the label map.

In [ ]:
from google.colab import files
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')